In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

from skimage.io import imread
from keras.applications.vgg16 import VGG16, preprocess_input
from skimage.transform import resize 
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import normalize

In [ ]:
# Liste für Trainingsbilder
trainImg = []

# Pfad zum Ordner mit den Bildern
folder_path = './ASL_Alphabet_Dataset/asl_alphabet_train/A/'

# Durchsuche alle Unterordner nach .jpg und .jpeg Dateien und füge sie zur Liste hinzu
for img_path in glob.glob(folder_path + '/*.jp*g', recursive=True):
    trainImg.append(resize(imread(img_path), (224, 224)))

# Konvertieren der Bilder in eine 2D-Array-Darstellung
image_array = np.array([img.flatten() for img in trainImg])

# Ausgabe der Anzahl der gefundenen Bilder
print(f"Anzahl der gefundenen Bilder: {len(trainImg)}")

In [ ]:
# Laden des VGG16-Modells, ohne die oberste Schicht (ohne Klassifizierungsschicht)
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Funktion zur Feature-Extraktion mit VGG16
def extract_features(img_array):
    img_array = preprocess_input(img_array)
    features = vgg16_model.predict(img_array)
    return features.reshape((len(img_array), -1))

# Feature-Extraktion mit VGG16
vgg16_features = extract_features(np.array(trainImg))

In [5]:
from sklearn.metrics import silhouette_score
from matplotlib.image import imsave
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KDTree

# Features normalisieren
normalized_features = StandardScaler().fit_transform(vgg16_features)

# Distanzschwellenwerte
distance_thresholds = [0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.30]

# Initialisieren der Variablen, um den besten Silhouetten-Score und die entsprechenden Parameter zu speichern
bester_silhouetten_score = -1
beste_leaf_size = None
bester_distanz_schwellenwert = None

for distanz_schwellenwert in distance_thresholds:
    # Clustering mit KDTree durchführen
    kdtree = KDTree(normalized_features, metric='euclidean')
    nachbarn = kdtree.query_radius(normalized_features, distanz_schwellenwert)
    labels = np.zeros(len(normalized_features), dtype=int)
    aktuelles_label = 1
    for i, nachbar_liste in enumerate(nachbarn):
        if labels[i] == 0:  # Punkt wurde noch keinem Cluster zugewiesen
            labels[nachbar_liste] = aktuelles_label
            aktuelles_label += 1

    # Silhouetten-Score berechnen
    silhouetten_score = silhouette_score(normalized_features, labels)

    # Ausgabe des aktuellen Silhouetten-Scores
    print(f"Distanzschwellenwert: {distanz_schwellenwert}, Silhouetten-Score: {silhouetten_score}")

    # Aktualisieren des besten Silhouetten-Scores und der entsprechenden Parameter, falls erforderlich
    if silhouetten_score > bester_silhouetten_score:
        bester_silhouetten_score = silhouetten_score
        bester_distanz_schwellenwert = distanz_schwellenwert

# Ausgabe der besten Parameterwerte und des entsprechenden Silhouetten-Scores
print("Beste Leaf-Größe:", beste_leaf_size)
print("Bester Distanzschwellenwert:", bester_distanz_schwellenwert)
print("Bester Silhouetten-Score:", bester_silhouetten_score)

# Ergebnisse für Verschiedene Parameter

## Leaf-size: 5

- Distanzschwellenwert: 0.1,    Silhouetten-Score: 0.7094
- Distanzschwellenwert: 0.2,    Silhouetten-Score: 0.7102
- Distanzschwellenwert: 0.3,    Silhouetten-Score: 0.7101
- Distanzschwellenwert: 0.4,    Silhouetten-Score: 0.7043
- Distanzschwellenwert: 0.5,    Silhouetten-Score: 0.6802
- Distanzschwellenwert: 0.6,    Silhouetten-Score: 0.6420
- Distanzschwellenwert: 0.7,    Silhouetten-Score: 0.6124
- Distanzschwellenwert: 0.8,    Silhouetten-Score: 0.5800
- Distanzschwellenwert: 0.9,    Silhouetten-Score: 0.5586
- Distanzschwellenwert: 1.0,    Silhouetten-Score: 0.5328

## Leaf-size: 10

- Distanzschwellenwert: 0.1,    Silhouetten-Score: 0.7094
- Distanzschwellenwert: 0.2,    Silhouetten-Score: 0.7102
- Distanzschwellenwert: 0.3,    Silhouetten-Score: 0.7101
- Distanzschwellenwert: 0.4,    Silhouetten-Score: 0.7043
- Distanzschwellenwert: 0.5,    Silhouetten-Score: 0.6802
- Distanzschwellenwert: 0.6,    Silhouetten-Score: 0.6420
- Distanzschwellenwert: 0.7,    Silhouetten-Score: 0.6124
- Distanzschwellenwert: 0.8,    Silhouetten-Score: 0.5800
- Distanzschwellenwert: 0.9,    Silhouetten-Score: 0.5586
- Distanzschwellenwert: 1.0,    Silhouetten-Score: 0.5328

...

## Leaf-size: 40

- Distanzschwellenwert: 0.1,    Silhouetten-Score: 0.7094
- Distanzschwellenwert: 0.2,    Silhouetten-Score: 0.7102
- Distanzschwellenwert: 0.3,    Silhouetten-Score: 0.7101
- Distanzschwellenwert: 0.4,    Silhouetten-Score: 0.7043
- Distanzschwellenwert: 0.5,    Silhouetten-Score: 0.6802
- Distanzschwellenwert: 0.6,    Silhouetten-Score: 0.6420
- Distanzschwellenwert: 0.7,    Silhouetten-Score: 0.6124
- Distanzschwellenwert: 0.8,    Silhouetten-Score: 0.5800
- Distanzschwellenwert: 0.9,    Silhouetten-Score: 0.5586
- Distanzschwellenwert: 1.0,    Silhouetten-Score: 0.5328

---

Leaf-size beeinflusst lediglich die Performance und nicht die Qualität der Cluster. Teste Distanz bis 2.0.

- Distanzschwellenwert: 1.1,    Silhouetten-Score: 0.5103
- Distanzschwellenwert: 1.2,    Silhouetten-Score: 0.4876
- Distanzschwellenwert: 1.3,    Silhouetten-Score: 0.4659
- Distanzschwellenwert: 1.4,    Silhouetten-Score: 0.4535
- Distanzschwellenwert: 1.5,    Silhouetten-Score: 0.4423
- Distanzschwellenwert: 1.6,    Silhouetten-Score: 0.4300
- Distanzschwellenwert: 1.7,    Silhouetten-Score: 0.4151
- Distanzschwellenwert: 1.8,    Silhouetten-Score: 0.4098
- Distanzschwellenwert: 1.9,    Silhouetten-Score: 0.3946
- Distanzschwellenwert: 2.0,    Silhouetten-Score: 0.3874

Teste Distanzschwelle 0.2 bis 0.3:

- Distanzschwellenwert: 0.20,    Silhouetten-Score: 0.7102
- Distanzschwellenwert: 0.21,    Silhouetten-Score: 0.7102
- Distanzschwellenwert: 0.22,    Silhouetten-Score: 0.7102
- Distanzschwellenwert: 0.23,    Silhouetten-Score: 0.7102
- Distanzschwellenwert: 0.24,    Silhouetten-Score: 0.7102
- Distanzschwellenwert: 0.25,    Silhouetten-Score: 0.7101
- Distanzschwellenwert: 0.26,    Silhouetten-Score: 0.7104
- Distanzschwellenwert: 0.27,    Silhouetten-Score: 0.7104
- Distanzschwellenwert: 0.28,    Silhouetten-Score: 0.7105
- Distanzschwellenwert: 0.29,    Silhouetten-Score: 0.7103
- Distanzschwellenwert: 0.30,    Silhouetten-Score: 0.7101

Beste Leaf-Größe: None <br>
Bester Distanzschwellenwert: 0.28 <br>
Bester Silhouetten-Score: 0.7105 <br>